### Criando ponto de montagem do container landing-zone

In [ ]:
# Conectando com Azure Data Lake usando SASToken
storageAccountName = "satcseguroimoveis"
sasToken = "sv=2022-11-02&ss=bfqt&srt=co&sp=rwdlacupyx&se=2024-06-28T09:15:32Z&st=2024-06-27T01:15:32Z&spr=https&sig=r6o07RmWkGJ8CZ0fCyK1jrMyLYgKljsBuU6VJ9iWUI4%3D"

def mount_adls(blobContainerName):
    try:
      dbutils.fs.mount(
        source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
        mount_point = f"/mnt/{storageAccountName}/{blobContainerName}",
        extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
      )
      print("OK!")
    except Exception as e:
      print("Falha", e)


mount_adls('landing-zone')
mount_adls('bronze')
mount_adls('silver')
mount_adls('gold')

OK!
OK!
OK!
OK!


### Lendo as tabelas do banco e salvando em .CSV no container landing-zone

In [ ]:
tables = ["apolice_cobertura", "cobertura", "sinistro", "pagamento", "apolice", "avaliacao", "imovel", "cliente"]

for table in tables:
    remote_table = (spark.read
    .format("sqlserver")
    .option("host", "satcseguroimoveissqlserver.database.windows.net")
    .option("user", "adminuser")
    .option("password", "Password12345!")
    .option("database", "satcseguroimoveisdatabase")
    .option("dbtable", f"dbo.{table}")
    .load())
    remote_table.write.parquet(f"/mnt/{storageAccountName}/landing-zone/{table}")

### Lendo os dados gravados

In [ ]:
# Lendo a tabela 'cliente' do ponto de montagem 'landing-zone'
cliente_df = spark.read.parquet(f"/mnt/satcseguroimoveis/landing-zone/cliente")

# Exibindo 50 resultados
display(cliente_df.limit(50))

id_cliente,nome,dt_nasc,endereco,telefone,email
1,Dr. Rhavi Barbosa,1992-01-19,"Rodovia de Melo, 74 São João Batista 73347810 Albuquerque / SP",+55 41 5800 2366,dr.rhavibarbosa@ramos.org
2,Evelyn Sá,1986-08-09,"Setor Ravi Lucca da Cruz, 94 Universo 77341-234 da Rocha / ES",+55 31 9959-5996,evelynsá@melo.com
3,Sra. Maysa Barros,1950-03-25,"Passarela Vasconcelos, 2 Baleia 89363-682 Pereira Grande / PB",+55 (011) 7038-6784,sra.maysabarros@oliveira.br
4,Maria Flor Barros,1966-08-19,Praça de Rodrigues Engenho Nogueira 32402-855 Nogueira Grande / PI,51 2645 3083,mariaflorbarros@garcia.br
5,Maria Vitória Silva,1936-04-09,"Residencial de Peixoto, 85 Pousada Santo Antonio 27512848 Azevedo de Fonseca / MA",+55 (071) 7619-7010,mariavitóriasilva@andrade.com
6,Lorenzo das Neves,1990-08-13,"Colônia Luiza Camargo, 53 Vila Puc 37660488 da Cruz / MG",+55 21 4755-3430,lorenzodasneves@silveira.net
7,Mathias Pastor,1948-02-12,"Travessa de Ramos, 29 São Gabriel 53620835 Nascimento / RR",61 3031 8052,mathiaspastor@moura.br
8,Ísis Cavalcanti,1996-02-26,"Conjunto da Conceição, 943 Petropolis 62202-846 Almeida / AM",+55 (031) 3385 4717,ísiscavalcanti@cunha.net
9,Stephany Teixeira,1971-03-03,"Área Gabrielly Borges, 364 Novo Tupi 12477804 Machado / ES",41 4006 4118,stephanyteixeira@fogaca.net
10,Rodrigo Santos,1960-10-25,"Feira Campos, 309 Vale Do Jatoba 42918-956 Marques / RJ",71 7236 4394,rodrigosantos@sousa.br
